# Tensorflow Approach

Code's Author: https://www.kaggle.com/code/realtimshady/water-bodies-segmentation-with-unet-and-tensorflow

Burada farklı bir kişiden almamın nedeni torch ile yazılmış bir kodun nasıl tensorflow'da yazılabilmesini ele almamdır. Buradaki değişikliklere ve iki kütüphanenin farklarını inceleyerek bazı konulardaki avantaj ve fırsatlarını görmek ana amacımdır...

In [ ]:
!kaggle datasets download -d franciscoescobar/satellite-images-of-water-bodies
from zipfile import ZipFile
import os
for i in os.listdir('C:/Users/mehmu/Desktop/PyProjects/Projects/github repo/SatelliteWaterBodies'):
    if '.zip' in i:
        print(i)
        with ZipFile(i, 'r') as zipObj:
            zipObj.extractall()
        os.remove(i)

In [ ]:
!pip install --upgrade opencv-contrib-python
!pip install segmentation-models-pytorch helper
!pip install -U git+https://github.com/albumentations-team/albumentations

Klasik import ve yüklemeler yapılıyor...

In [1]:
from functools import partial
from glob import glob
from tqdm import tqdm
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage.filters import threshold_otsu
from tensorflow.keras.metrics import MeanIoU
from tensorflow import keras
from tensorflow.data import Dataset, AUTOTUNE

## Data Loader

Dosyaların yollarını doğru girip kontrol etmek burada önemli olabiliyor.

In [3]:
images_dir = 'C:/Users/mehmu/Desktop/PyProjects/Projects/github repo/SatelliteWaterBodies/Water Bodies Dataset/Images'
masks_dir = 'C:/Users/mehmu/Desktop/PyProjects/Projects/github repo/SatelliteWaterBodies/Water Bodies Dataset/Masks'

dirname, _, filenames = next(os.walk(images_dir))

Alt kısımda normalde torch'ta küçük kodlarla yapılabilecek işleri fonksiyona çevirme işlemlerini görüyorsunuz.

Her ne kadar torch öğrenmesi ve uygulaması zor gibi gözükse de bazen daha basit yerleri olabiliyor. Fakat bu kodlar saklanıp ilerideki projelerde kullanılıp işlerin kolaylaştırılması ve kütüphane gibi kullanılarak yazılanlar azaltılabilir.

In [4]:
@tf.function
def load_img_with_mask(image_path, images_dir: str = 'Images', masks_dir: str = 'Masks',images_extension: str = 'jpg', masks_extension: str = 'jpg') -> dict:
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)

    mask_filename = tf.strings.regex_replace(image_path, images_dir, masks_dir)
    mask_filename = tf.strings.regex_replace(mask_filename, images_extension, masks_extension)
    mask = tf.io.read_file(mask_filename)
    mask = tf.image.decode_image(mask, channels=1, expand_animations = False)
    return (image, mask)

@tf.function
def resize_images(images, masks, max_image_size=2000):
    shape = tf.shape(images)
    scale = (tf.reduce_max(shape) // max_image_size) + 1
    target_height, target_width = shape[-3] // scale, shape[-2] // scale
    images = tf.cast(images, tf.float32)
    masks = tf.cast(masks, tf.float32)
    if scale != 1:
        images = tf.image.resize(images, (target_height, target_width), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        masks = tf.image.resize(masks, (target_height, target_width), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return (images, masks)

@tf.function
def scale_values(images, masks, mask_split_threshold = 128):
    images = tf.math.divide(images, 255)
    masks = tf.where(masks > mask_split_threshold, 1, 0)
    return (images, masks)

@tf.function
def pad_images(images, masks, pad_mul=16, offset=0):
    shape = tf.shape(images)
    height, width = shape[-3], shape[-2]
    target_height = height + tf.math.floormod(tf.math.negative(height), pad_mul)
    target_width = width + tf.math.floormod(tf.math.negative(width), pad_mul)
    images = tf.image.pad_to_bounding_box(images, offset, offset, target_height, target_width)
    masks = tf.cast(tf.image.pad_to_bounding_box(masks, offset, offset, target_height, target_width), tf.uint8)
    return (images, masks)

@tf.function
def augment(images, masks):
    if np.random.choice(2):
        images = tf.image.flip_left_right(images)
        masks = tf.image.flip_left_right(masks)
    if np.random.choice(2):
        images = tf.image.flip_up_down(images)
        masks = tf.image.flip_up_down(masks)
    if np.random.choice(2):
        images = tf.image.adjust_brightness(images)
        masks = tf.image.adjust_brightness(masks)
    if np.random.choice(2):
        images = tf.image.adjust_contrast(images)
        masks = tf.image.adjust_contrast(masks)
    if np.random.choice(2):
        images = tf.image.adjust_saturation(images)
        masks = tf.image.adjust_saturation(masks)
    

    return (images, masks)

Üstteki kodlardan bahsetmek gerekirse;

**load_img_with_mask**: Normal fotoğraf ve mask'i birleştirme işlemidir.

**resize_images**: Fotoğrafları boyutunu istediğimiz hale getiriyoruz.

**scale_values**: Değerleri 255 ile scale ediyoruz.

**pad_images**: Fotoğraflara padding ekleme işlemidir.

**augment**: Sağa sola, aşağı yukarı fotoğrafları döndürme işlemidir.

In [6]:
train_split = 0.8
test_split = 0.2

sample_list = sorted(glob('C:/Users/mehmu/Desktop/PyProjects/Projects/github repo/SatelliteWaterBodies/Water Bodies Dataset/Images/*.jpg'))

n_samples = len(sample_list)
train_split = round(train_split*n_samples)
test_split = round(test_split*n_samples)

train_list = sample_list[0:train_split]
valid_list = sample_list[train_split:train_split+test_split]
test_list = sample_list[train_split+test_split:n_samples]

Üstte fotoğraflar yüklenip bölmek için belirlediğimiz oranda bölünüyor.

Altta ise üstte belirlenen fonksiyonlar teker teker kullanılarak eğitime hazır bir şekle getiriliyor...

In [7]:
test_dataset = Dataset.list_files(test_list)
test_dataset = test_dataset.map(load_img_with_mask, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(scale_values, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.shuffle(20)
test_dataset = test_dataset.map(lambda img, mask: resize_images(img, mask, max_image_size=2500), num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(pad_images, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(1).prefetch(buffer_size=AUTOTUNE)



validation_dataset = Dataset.list_files(valid_list)
validation_dataset = validation_dataset.map(load_img_with_mask, num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.map(scale_values, num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.shuffle(20)
validation_dataset = validation_dataset.map(resize_images, num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.map(pad_images, num_parallel_calls=AUTOTUNE).cache('cache')
validation_dataset = validation_dataset.map(augment, num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.batch(1).prefetch(buffer_size=AUTOTUNE)



train_dataset = Dataset.list_files(train_list)
train_dataset = train_dataset.map(load_img_with_mask, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(scale_values, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.shuffle(20)
train_dataset = train_dataset.map(resize_images, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(pad_images, num_parallel_calls=AUTOTUNE).cache('cache')
train_dataset = train_dataset.map(augment, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(1).prefetch(buffer_size=AUTOTUNE)

## Model

Unet mimarisini kendimiz yapıyoruz. Burada torch'ta yapıldığı gibi direkt de alınabilir bir fonksiyon yazılabilirmiş.

In [8]:
def get_unet(hidden_activation='relu', initializer='he_normal', output_activation='sigmoid'):
    PartialConv = partial(keras.layers.Conv2D,
        activation=hidden_activation,
        kernel_initializer=initializer,      
        padding='same')
    
    # Encoder
    model_input = keras.layers.Input(shape=(None, None, 3))
    enc_cov_1 = PartialConv(32, 3)(model_input)
    enc_cov_1 = PartialConv(32, 3)(enc_cov_1)
    enc_pool_1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(enc_cov_1)
    
    enc_cov_2 = PartialConv(64, 3)(enc_pool_1)
    enc_cov_2 = PartialConv(64, 3)(enc_cov_2)
    enc_pool_2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(enc_cov_2)
    
    enc_cov_3 = PartialConv(128, 3)(enc_pool_2)
    enc_cov_3 = PartialConv(128, 3)(enc_cov_3)
    enc_pool_3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(enc_cov_3)
    
    # Center
    center_cov = PartialConv(256, 3)(enc_pool_3)
    center_cov = PartialConv(256, 3)(center_cov)
    
    # Decoder
    upsampling1 = keras.layers.UpSampling2D(size=(2, 2))(center_cov)
    dec_up_conv_1 = PartialConv(128, 2)(upsampling1)
    dec_merged_1 = tf.keras.layers.Concatenate(axis=3)([enc_cov_3, dec_up_conv_1])
    dec_conv_1 = PartialConv(128, 3)(dec_merged_1)
    dec_conv_1 = PartialConv(128, 3)(dec_conv_1)
    
    upsampling2 = keras.layers.UpSampling2D(size=(2, 2))(dec_conv_1)
    dec_up_conv_2 = PartialConv(64, 2)(upsampling2)
    dec_merged_2 = tf.keras.layers.Concatenate(axis=3)([enc_cov_2, dec_up_conv_2])
    dec_conv_2 = PartialConv(64, 3)(dec_merged_2)
    dec_conv_2 = PartialConv(64, 3)(dec_conv_2)
    
    upsampling3 = keras.layers.UpSampling2D(size=(2, 2))(dec_conv_2)
    dec_up_conv_3 = PartialConv(32, 2)(upsampling3)
    dec_merged_3 = tf.keras.layers.Concatenate(axis=3)([enc_cov_1, dec_up_conv_3])
    dec_conv_3 = PartialConv(32, 3)(dec_merged_3)
    dec_conv_3 =  PartialConv(32, 3)(dec_conv_3)
    
    output = keras.layers.Conv2D(1, 1, activation=output_activation)(dec_conv_3)
    
    return tf.keras.Model(inputs=model_input, outputs=output)

## Eğitim

Modelimizi atıyoruz ve Nadam optimizörünü seçek compile işlemini başlatıyoruz. 

In [9]:
model = get_unet()

optimizer = tf.keras.optimizers.Nadam()
model.compile(loss='binary_crossentropy', optimizer=optimizer)

*val_loss*'u izleyip değerlerine göre durdurmak için early_stopping kullanıyoruz.

*lr_reduce* ile eğitim ilerledikçe learning rate'i düşürüyoruz.


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=1)

epochs = 100
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=epochs, callbacks=[early_stopping, lr_reduce])

## Kontrol ve Test

In [ ]:
def OtsuFilter(prediction, bias=0):
    threshold = threshold_otsu(prediction) - bias
    
    return prediction > threshold

In [ ]:
n_examples = 5

fig, axs = plt.subplots(n_examples, 3, figsize=(14, n_examples*7), constrained_layout=True)
for ax, ele in zip(axs, test_dataset.take(n_examples)):
    image, y_true = ele
    prediction = model.predict(image)[0]
    prediction = OtsuFilter(prediction)
    ax[0].set_title('Original image')
    ax[0].imshow(image[0])
    ax[1].set_title('Original mask')
    ax[1].imshow(y_true[0])
    ax[2].set_title('Predicted area')
    ax[2].imshow(prediction)

In [ ]:
meanIoU = MeanIoU(num_classes=2)
for (image, y_true) in tqdm(test_dataset.take(test_split)):
    prediction = model.predict(image)[0]
    prediction = OtsuFilter(prediction)
    meanIoU.update_state(y_true[0], prediction)
print(meanIoU.result().numpy())

In [ ]:
model.save('waternet.h5')